<a href="https://colab.research.google.com/github/TheAbhishekraj/Spoon_fork_detection/blob/main/Data%20Collection%20Sample%20/Fork_Image_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image Generetor Fork

In [ ]:
# ==============================================================================
# Step 1: Install necessary libraries (if in a new session)
# ==============================================================================
# This step is only needed once per session. If you are in the same notebook
# and have already run this for spoons, you can optionally comment it out.
print("Installing required libraries... This may take a minute.")
!pip install diffusers transformers accelerate invisible_watermark --quiet
!pip install xformers --quiet # Helps with performance and memory

# ==============================================================================
# Step 2: Import libraries and set up the image generation pipeline
# ==============================================================================
import torch
from diffusers import StableDiffusionPipeline
import os
import shutil
from tqdm.auto import tqdm # For a nice progress bar

print("Setting up the image generation pipeline...")

# Load the pre-trained Stable Diffusion model.
# This will use the cached model if you've already downloaded it.
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Move the pipeline to the GPU for faster processing.
pipe = pipe.to("cuda")

# ==============================================================================
# Step 3: Define the list of prompts for the 50 fork images
# ==============================================================================
# These are new descriptions tailored specifically for forks.
fork_prompts = [
    # Dinner Forks (Metal)
    "fork_01.jpg: A standard stainless steel dinner fork, top-down view on a pure white background, bright studio lighting, photorealistic.",
    "fork_02.jpg: A shiny silver fork with four tines, side profile, resting on a dark granite countertop, sharp reflections, 4k.",
    "fork_03.jpg: A single dinner fork at a 45-degree angle on a rustic, weathered wooden table, natural light from a window.",
    "fork_04.jpg: Macro close-up shot of the tines of a metal fork, showing fine details and texture, blurred background.",
    "fork_05.jpg: An ornate, vintage silver fork with detailed engravings on the handle, placed on a black velvet cloth.",
    "fork_06.jpg: A modern, matte-finish black metal fork next to a white ceramic plate, minimalist style.",
    "fork_07.jpg: A fork resting on a folded white linen napkin next to a dinner plate, viewed from above.",
    "fork_08.jpg: A slightly tarnished brass fork on a cool marble surface, interesting textures and patina.",
    "fork_09.jpg: A fork and knife set, crossed over each other on a plate, elegant dining composition.",
    "fork_10.jpg: A fork piercing a single piece of cooked penne pasta, action shot, high detail.",
    # Salad/Dessert Forks (Metal & Plastic)
    "fork_11.jpg: A small, polished silver dessert fork resting next to a slice of chocolate cake on a plate.",
    "fork_12.jpg: A bright green plastic fork lying on a colorful, patterned picnic blanket.",
    "fork_13.jpg: A salad fork with wider outer tines, next to a bowl of fresh green salad.",
    "fork_14.jpg: A small, three-pronged cocktail fork (oyster fork) next to an empty cocktail glass.",
    "fork_15.jpg: A disposable, translucent plastic fork on a plain brown paper napkin from a takeout bag.",
    "fork_16.jpg: A decorative dessert fork with a ceramic handle, placed on a small saucer.",
    "fork_17.jpg: A simple metal fork, shot from a low angle on a clean, light-blue background, isolated.",
    "fork_18.jpg: Three forks of different types (dinner, salad, dessert) arranged neatly in a row.",
    "fork_19.jpg: A fork twirling a small amount of spaghetti in a white bowl, viewed from above.",
    "fork_20.jpg: A red plastic fork on a white background, isolated, product photography style.",
    # Specialty & Serving Forks
    "fork_21.jpg: A large, two-pronged carving fork with a wooden handle, resting on a wooden cutting board.",
    "fork_22.jpg: A large metal serving fork resting in an empty glass salad bowl, bright and clean.",
    "fork_23.jpg: A fondue fork with a colored tip, dipped in melted chocolate, macro photography.",
    "fork_24.jpg: A spaghetti fork with specially designed tines, isolated on a white background.",
    "fork_25.jpg: A large serving fork resting on the edge of a stainless steel pot of cooked vegetables.",
    "fork_26.jpg: A small, elegant pastry fork with a notched tine, next to a croissant.",
    "fork_27.jpg: A barbecue fork with very long tines and a long handle, leaning against a grill.",
    "fork_28.jpg: A rustic, hand-forged iron fork on a dark, moody background, historical style.",
    "fork_29.jpg: A serving fork and spoon set made of acacia wood, side-by-side on a tablecloth.",
    "fork_30.jpg: A fish fork with a wide, flat shape, next to a piece of grilled salmon.",
    # Plastic Forks
    "fork_31.jpg: A standard white disposable plastic fork, isolated on a bright yellow background, pop art style.",
    "fork_32.jpg: A sturdy, reusable black plastic fork on a kitchen countertop.",
    "fork_33.jpg: A small, light-blue plastic fork for a child, on a high-chair tray.",
    "fork_34.jpg: A fan arrangement of five plastic forks in different vibrant colors (red, blue, green, yellow, orange).",
    "fork_35.jpg: A clear plastic fork from a takeout salad box, shot in a close-up.",
    "fork_36.jpg: A single white plastic fork piercing a cherry tomato, on a paper plate.",
    "fork_37.jpg: A bundle of new, wrapped plastic forks from a party supply store.",
    "fork_38.jpg: Close-up shot showing the recycling symbol on the back of a plastic fork.",
    "fork_39.jpg: A plastic fork with one of its tines bent out of shape, on a dark surface.",
    "fork_40.jpg: A heavy-duty gray plastic fork, camping gear style, next to a metal plate.",
    # Artistic & Conceptual Shots
    "fork_41.jpg: A fork standing upright, stuck into a piece of fruit like an apple.",
    "fork_42.jpg: The shadow of a fork cast on a white wall, creating long, dramatic lines.",
    "fork_43.jpg: A fork with its tines bent into a creative, abstract shape, artistic photography.",
    "fork_44.jpg: A fork seen through a glass of water, causing a distorted refraction effect.",
    "fork_45.jpg: A collection of many different vintage forks arranged in a pattern, top-down view.",
    "fork_46.jpg: Water splashing over the tines of a silver fork, high-speed photography.",
    "fork_47.jpg: A single fork lying on a sandy beach.",
    "fork_48.jpg: A copper-colored metal fork giving off warm reflections, on a dark slate platter.",
    "fork_49.jpg: A fork balancing perfectly on its edge on a table.",
    "fork_50.jpg: A close-up of a fork's tines holding a single green pea.",
]

# ==============================================================================
# Step 4: Generate the images and save them to a directory
# ==============================================================================
output_dir = "generated_forks"  # Changed directory name
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Starting image generation for forks. This will create {len(fork_prompts)} images...")

# Loop through each prompt, generate an image, and save it
for prompt in tqdm(fork_prompts, desc="Generating Fork Images"):
    # Split the prompt to get the filename and the actual description
    filename, text_prompt = prompt.split(": ", 1)

    # Generate the image
    image = pipe(text_prompt).images[0]

    # Save the image
    image_path = os.path.join(output_dir, filename)
    image.save(image_path)

print("All fork images have been generated successfully!")

# ==============================================================================
# Step 5: Zip the generated images
# ==============================================================================
zip_filename_base = "fork_images"  # Changed zip filename
zip_path = f"{zip_filename_base}.zip"

print(f"Creating zip file: {zip_path}...")

# Use shutil to create a zip archive of the output directory
shutil.make_archive(zip_filename_base, 'zip', output_dir)

print("="*60)
print("✅ DONE! ✅")
print(f"Your zip file '{zip_path}' is ready.")
print("Please refresh the file browser on the left to see and download it.")
print("="*60)

Installing required libraries... This may take a minute.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.5 MB/s eta 0:00:00
Setting up the image generation pipeline...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

All fork images have been generated successfully!
Creating zip file: fork_images.zip...
✅ DONE! ✅
Your zip file 'fork_images.zip' is ready.
Please refresh the file browser on the left to see and download it.
